In [127]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from pathlib import Path
import os
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, recall_score
root = Path('')

In [75]:
encoder = {'left': 0, 'center': 1, 'right': 2}
classes = list(encoder.keys())

### Best model article deberta-v3

In [62]:
X_train_article = pd.read_pickle(root / 'probability/deberta-v3-base_train_prob_bias_article.pkl')
X_test_article = pd.read_pickle(root / 'probability/deberta-v3-base_test_prob_bias_article.pkl')

### Best model description bert

In [63]:
X_train_des = pd.read_pickle(root / 'probability/bert-base-uncased_train_prob_bias_description.pkl')
X_test_des = pd.read_pickle(root / 'probability/bert-base-uncased_test_prob_bias_description.pkl')

In [108]:
def merge_dataset(df, df_1):
    df = df.merge(df_1, on=['website', 'target'])
    df['left'] = df['left_x'] + df['left_y']
    df['right'] = df['right_x'] + df['right_y']
    df['center'] = df['center_x'] + df['center_y']
    return df[['target', 'website'] + classes]

In [109]:
X_train = merge_dataset(X_train_article, X_train_des)
X_test = merge_dataset(X_test_article, X_test_des)

In [110]:
clf = LogisticRegression()
clf.fit(X_train[classes], X_train['target'])
y_pred = clf.predict(X_test[classes])
y_test = X_test['target']
accuracy = accuracy_score(y_test, y_pred)
macro_f1 = f1_score(y_test, y_pred, average='macro')
avg_recall = recall_score(y_test, y_pred, average='macro')
print("Accuracy:", accuracy)
print("Macro-F1 Score:", macro_f1)
print("Average Recall:", avg_recall)

Accuracy: 0.9298245614035088
Macro-F1 Score: 0.9263511101592061
Average Recall: 0.9266663237722952


### Step 3

In [157]:
def read_models_file(X_train, num_model='M1'):
    files = []
    renames = {'logit_1': 'center', 'logit_2':'right', 'logit_3':'left'}
    for file in os.listdir(root / 'step_3' / num_model):
        if file.endswith('csv'):
            df = pd.read_csv(root / 'step_3' / num_model / file)
            df.rename(columns=renames, inplace=True)
            df['target'] = None
            for index, row in df.iterrows():
                df.iloc[index, -1] = row.index[row[1:].argmax() + 1]
            df = pd.concat([X_train, df])
            files.append(df)
    
    return files

def evaluate(files: list[pd.DataFrame], X_test: pd.DataFrame, params_log_reg: dict):
    accuracy_list = []
    macro_f1_list = []
    avg_recall_list = []
    for index, X_train in enumerate(files):
        clf = LogisticRegression(**params_log_reg)
        clf.fit(X_train[classes], X_train['target'])
        y_pred = clf.predict(X_test[classes])
        y_test = X_test['target']
        accuracy = accuracy_score(y_test, y_pred)
        macro_f1 = f1_score(y_test, y_pred, average='macro')
        avg_recall = recall_score(y_test, y_pred, average='macro')
        accuracy_list.append(accuracy), macro_f1_list.append(macro_f1), avg_recall_list.append(avg_recall)
        print(f'Index: {index}')
        print("Accuracy:", accuracy)
        print("Macro-F1 Score:", macro_f1)
        print("Average Recall:", avg_recall)
    accuracy_std = np.std(accuracy_list)
    macro_f1_std = np.std(macro_f1_list)
    avg_recall_std = np.std(avg_recall_list)

    print(f"Standard Deviation of Accuracy: {accuracy_std}")
    print(f"Standard Deviation of Macro-F1 Score: {macro_f1_std}")
    print(f"Standard Deviation of Average Recall: {avg_recall_std}")
    print()
        

#### Model m1

In [188]:
files = read_models_file(X_train, num_model='M1')
params = {
    'C': 2, 'penalty': 'l1',
    'solver': 'liblinear',
}
evaluate(files, X_test, params)

Index: 0
Accuracy: 0.9298245614035088
Macro-F1 Score: 0.9260152632883744
Average Recall: 0.9246965861055391
Index: 1
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 2
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 3
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 4
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Standard Deviation of Accuracy: 0.002339181286549685
Standard Deviation of Macro-F1 Score: 0.0023812431487829854
Standard Deviation of Average Recall: 0.00340227415166976



Accuracy: 0.9345029240 ± 0.0023
Macro-F1 Score: 0.9307777496 ± 0.0024
Average Recall: 0.9315011344 ± 0.0034

#### Model m2

In [189]:
files = read_models_file(X_train, num_model='M2')
params = {
    'C': 2, 'penalty': 'l1',
    'solver': 'liblinear',
}
evaluate(files, X_test, params)

Index: 0
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9309296911244568
Average Recall: 0.9332022714847135
Index: 1
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 2
Accuracy: 0.9298245614035088
Macro-F1 Score: 0.9252816180235536
Average Recall: 0.9266663237722952
Index: 3
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 4
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Standard Deviation of Accuracy: 0.002339181286549685
Standard Deviation of Macro-F1 Score: 0.002602116837811581
Standard Deviation of Average Recall: 0.0026143790849673105



Accuracy: 0.9345029240 ± 0.0023
Macro-F1 Score: 0.9304232845 ± 0.0026
Average Recall: 0.9318950819 ± 0.0026

#### Model m3

In [190]:
files = read_models_file(X_train, num_model='M3')
params = {
    'C': 2, 'penalty': 'l1',
    'solver': 'liblinear',
}
evaluate(files, X_test, params)

Index: 0
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 1
Accuracy: 0.9298245614035088
Macro-F1 Score: 0.9260152632883744
Average Recall: 0.9246965861055391
Index: 2
Accuracy: 0.9239766081871345
Macro-F1 Score: 0.9195986994096005
Average Recall: 0.9201303760598769
Index: 3
Accuracy: 0.935672514619883
Macro-F1 Score: 0.9319683711603318
Average Recall: 0.9332022714847135
Index: 4
Accuracy: 0.9298245614035088
Macro-F1 Score: 0.9260152632883744
Average Recall: 0.9246965861055391
Standard Deviation of Accuracy: 0.004376207469911036
Standard Deviation of Macro-F1 Score: 0.004604867493438223
Standard Deviation of Average Recall: 0.005187817203548877

